In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import yfinance as yf

from yahoo_fin import stock_info as si

from pandas_datareader import data

In [2]:
def cumm_prod(price_df, start=None, end=None):
    """
    Calculating the cummulative product of the percentage change to normalize the return and be
    able to compare the stocks. 

    """
    if start == None and end == None:
        return_series = (price_df.pct_change()+1).cumprod() - 1
        return_series.dropna(inplace=True)
    else:
        if end == None:
            end = date.today()
        return_series = (price_df.loc[start:end].pct_change()+1).cumprod() - 1
        return_series.dropna(inplace=True)
        
    
    return return_series

def annualized_return(adjClose_return_df, n_years=None):
    """
    n_years needs to represent the period of the adjClose_return_df.
        If it's 1 year worth of cummulative return, you input 1 for the 1y annualized returns.
        If it's 3 years worth of cummulative return, you input 3 for the long period annualized returns.

        (1+total return)pow(1/N) - 1) where N = number of years
    """
    if n_years == None or n_years == 0:
        print("Input How many years does your data covers!")
    else:
        annualized_return = ((1 + adjClose_return_df.tail(1))**(1/n_years)-1) * 100
        return annualized_return

In [3]:
def market_cap_cat(market_cap):
    try:
        if market_cap > 10000000000:
            market_cap_cat = 'Large'
        elif 2000000000 < market_cap < 1000000000:
            market_cap_cat = 'Medium'
        else:
            market_cap_cat = 'Small'
        return market_cap_cat
    except:
        return 'NaN'

def earnings_expectations(FPE, TPE):
    try:
        if FPE == None or TPE == None:
            return 'NaN'
        else:
            if FPE > TPE:
                expect = 'Decrease'
            else:
                expect = 'Increase'
                
            return expect
    except:
        return 'NaN'

def CAPM_analysis(beta, annual_return, market_return, risk_free_rate = 0.0192):
    try:
        if beta == None or annual_return == None or market_return == None:
            return 'NaN'
        else:
            capm_status = 'TBD'
            
            capm_ER = risk_free_rate + beta * (market_return - risk_free_rate)

            if annual_return > capm_ER:
                capm_status = 'OverValued'
            else:
                capm_status = 'UnderValued'
        
        return capm_status, capm_ER
    except:
        return 'NaN'
        
def intrinsic_valuation(ticker, eps_ttm, growth_estimate_5y, PE_trailing, target_return, safety_margin=0.5):

    try:
        if ticker == None or eps_ttm == None or growth_estimate_5y == None or PE_trailing == None or target_return == None:
            return 'NaN'
        else:
            irv_status = 'TBD'

            forecast_eps_at_year_10 = eps_ttm * ( 1 + growth_estimate_5y) ** 9
            forecast_value_at_year_10 = forecast_eps_at_year_10 * PE_trailing

            pv_future_value = forecast_value_at_year_10/(1 + target_return)**9

            pv_forecast_value_safe = pv_future_value * safety_margin

            if float(si.get_live_price(ticker)) > pv_forecast_value_safe:
                irv_status = 'Above Fair Price'
            else:
                irv_status = 'Below Fair Price'

            return irv_status, pv_forecast_value_safe
    except:
        return 'NaN'

def get_ESG(ticker):
    ticket = yf.Ticker(ticker)
    esg_data = pd.DataFrame(ticket.sustainability)['Value'][['socialScore', 'governanceScore', 'environmentScore', 'totalEsg', 'percentile']]
    
    return esg_data.to_dict()

def esg_status(esg_Rscore):
    if esg_Rscore == None:
        return 'NaN'
    else:
        if float(esg_Rscore) < 20:
            return 'Sustainable'
        elif 21 < float(esg_Rscore) < 30:
            return 'Average'
        else:
            return 'Not Sustainable'

In [4]:
SPY_adj_close = si.get_data('SPY', start_date= str(date.today().year - 10) +'-'+ str(date.today().month) +'-'+ str(date.today().day), end_date=str(date.today()))['adjclose']
cummulative_prod_SPY = cumm_prod(SPY_adj_close)
annu_return_10y_SPY = annualized_return(cummulative_prod_SPY, 10)[0]

In [5]:
def get_data(ticker):

    info = {}

    info['ticker'] = ticker
    info['name'] = si.get_quote_data(ticker)['shortName']
    info['sector'] = si.get_company_info(ticker)['Value']['sector']

    adj_close = si.get_data(ticker, start_date= str(date.today().year - 10) +'-'+ str(date.today().month) +'-'+ str(date.today().day), end_date=str(date.today()))['adjclose']
    cummulative_prod = cumm_prod(adj_close)
    annu_return_10y = annualized_return(cummulative_prod, 10)[0]

    info['current_price'] = float(si.get_live_price(ticker))
    info['annu_return_10y'] = annu_return_10y

    info['market_cap'] = si.get_quote_table(ticker)['Market Cap']

    cap = float(si.get_quote_data('AMD')['marketCap'])
    info['marketcap_cat'] = market_cap_cat(cap)


    info['beta'] = float(si.get_quote_table(ticker)['Beta (5Y Monthly)'])

    esg = get_ESG(ticker)
    info['socialScore'] = esg['socialScore']
    info['governanceScore'] = esg['governanceScore']
    info['environmentScore'] = esg['environmentScore']
    info['ESG_risk'] = esg['totalEsg']
    info['percentile'] = esg['percentile']

    info['esg_status'] = esg_status(esg['totalEsg'])
    

    info['eps_ttm'] = float(si.get_quote_table(ticker)['EPS (TTM)'])
    info['growth_estimate_5y'] = float(si.get_analysts_info(ticker)['Growth Estimates'][ticker][4][:-1]) / 100

    # if FPE > TPE => Investor expect a decrease in earnings.
    info['pe_forward'] = float(si.get_quote_data(ticker)['forwardPE'])
    info['pe_trailing'] = float(si.get_quote_data(ticker)['trailingPE'])

    info['Earnings_expectation'] = earnings_expectations(info['pe_forward'], info['pe_trailing'])
    info['irv_FairValue'] = intrinsic_valuation(ticker, info['eps_ttm'],  info['growth_estimate_5y'], info['pe_trailing'], 0.15)[1]
    info['irv_status'] = intrinsic_valuation(ticker, info['eps_ttm'],  info['growth_estimate_5y'], info['pe_trailing'], 0.15)[0]

    info['capm_status'] = CAPM_analysis(info['beta'], annu_return_10y, annu_return_10y_SPY)[0]



    return info

In [6]:
panel_data = pd.DataFrame(columns=['ticker', 'name', 'sector', 'Earnings_expectation', 'irv_status', 'capm_status', 'esg_status', 'current_price', 'annu_return_10y', 'market_cap', 'marketcap_cat', 'beta', 'eps_ttm', 'growth_estimate_5y', 'pe_forward', 'pe_trailing', 'irv_FairValue', 'socialScore', 'governanceScore', 'environmentScore', 'ESG_risk', 'percentile'])

In [7]:
sp500_list_50 = si.tickers_sp500()

In [8]:

for ticker in sp500_list_50:
        try:
                print(ticker)
                info = get_data(ticker)
                panel_data = panel_data.append(info,ignore_index=True)
        except:
                pass
       



A
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
ANTM
AON
AOS
APA
APD
APH
APTV
ARE
ATO
ATVI
AVB
AVGO
AVY
AWK
AXP
AZO
BA
BAC
BAX
BBWI
BBY
BDX
BEN
BF-B
BIIB
BIO
BK
BKNG
BKR
BLK
BLL
BMY
BR
BRK-B
BRO
BSX
BWA
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDAY
CDNS
CDW
CE
CEG
CERN
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COST
CPB
CPRT
CRL
CRM
CSCO
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CTXS
CVS
CVX
CZR
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
DISCA
DISCK
DISH
DLR
DLTR
DOV
DOW
DPZ
DRE
DRI
DTE
DUK
DVA
DVN
DXC
DXCM
EA
EBAY
ECL
ED
EFX
EIX
EL
EMN
EMR
ENPH
EOG
EPAM
EQIX
EQR
ES
ESS
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FB
FBHS
FCX
FDS
FDX
FE
FFIV
FIS
FISV
FITB
FLT
FMC
FOX
FOXA
FRC
FRT
FTNT
FTV
GD
GE
GILD
GIS
GL
GLW
GM
GNRC
GOOG
GOOGL
GPC
GPN
GRMN
GS
GWW
HAL
HAS
HBAN
HCA
HD
HES
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRL
HSIC
HST
HSY
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILM

In [9]:
panel_data

,ticker,name,sector,Earnings_expectation,irv_status,capm_status,esg_status,current_price,annu_return_10y,market_cap,...,eps_ttm,growth_estimate_5y,pe_forward,pe_trailing,irv_FairValue,socialScore,governanceScore,environmentScore,ESG_risk,percentile
0,A,"Agilent Technologies, Inc.",Healthcare,Increase,Above Fair Price,OverValued,Sustainable,132.050003,16.852985,39.663B,...,3.94,0.1231,28.215815,33.515230,53.357284,8.65,6.34,0.34,15.33,10.48
1,AAP,Advance Auto Parts Inc.,Consumer Cyclical,Increase,Above Fair Price,UnderValued,Sustainable,218.800003,10.299689,13.368B,...,9.55,0.1525,14.235524,22.910995,111.559143,9.17,3.70,0.10,12.97,5.74
2,AAPL,Apple Inc.,Technology,Increase,Above Fair Price,OverValued,Sustainable,167.300003,26.658995,2.73T,...,6.01,0.1485,25.503050,27.813800,82.604411,6.86,8.76,0.65,16.27,12.87
3,ABBV,AbbVie Inc.,Healthcare,Increase,Above Fair Price,OverValued,Average,144.029999,19.632876,254.628B,...,6.45,0.0410,11.738386,22.330233,29.389959,16.92,9.98,1.12,28.01,53.78
4,ABC,AmerisourceBergen Corporation,Healthcare,Increase,Above Fair Price,OverValued,Sustainable,141.500000,16.451440,29.593B,...,7.71,0.1034,12.229905,18.348030,48.745000,6.35,5.50,0.84,12.69,5.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,XRAY,DENTSPLY SIRONA Inc.,Healthcare,Increase,Below Fair Price,UnderValued,Not Sustainable,55.090000,4.236675,12.043B,...,1.90,0.2497,17.269592,28.964249,58.151007,10.42,6.55,3.15,20.12,24.40
397,XYL,Xylem Inc.,Industrials,Increase,Above Fair Price,UnderValued,Sustainable,90.660004,14.126965,16.348B,...,2.55,0.1876,30.220001,35.566890,60.576924,7.58,5.08,3.30,15.95,12.05
398,YUM,"Yum! Brands, Inc.",Consumer Cyclical,Decrease,Above Fair Price,UnderValued,Average,125.800003,12.452702,36.876B,...,5.18,0.1521,25.414143,24.295095,63.966029,13.08,4.70,3.90,21.68,29.73
399,ZBH,"Zimmer Biomet Holdings, Inc.",Healthcare,Increase,Above Fair Price,UnderValued,Average,121.019997,7.981383,25.282B,...,3.91,0.0975,15.089774,30.919773,39.695681,13.62,9.93,4.53,28.08,54.02


In [10]:
panel_data.to_csv('sample_stock_valuation_2.csv', index=False)